Google Colab to work with Spotify API and retrieving lyrics, tempo and time signature

In [ ]:
!pip3 install syrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 9.7 MB/s eta 0:00:00
  Created wheel for tinytag: filename=tinytag-1.9.0-py3-none-any.whl size=36820 sha256=500c8a77d305daae4638b9d5c4236e18b846af7e986f065905ab64fdcd00074d
  Stored in directory: /root/.cache/pip/wheels/c5/30/dd/4838f2a4769fbe88d28f55035ed2f3f52be5e30ea76e4d656f
Successfully built tinytag


In [ ]:
import os
import base64
from requests import post,get
import json
import webbrowser

In [ ]:
# From https://developer.spotify.com/ create new application. Dashboard, sign in. (Prova2)
#API that queries information about spotify library
#Client credential work flow: 
#Request an acces token: sending client_in, client_secret, grant_type .... 
#To spotify account service: returns a temporary access_token. 
#Having the access token, we can send requests to spotify web api, returns JSON objects



client_id ='cc1298c00eee4bf8a344bfb2056ac496'
client_secret = '27d6b03a2d8048e2bb48fe2d55d78aae'

#retrieve auth token - return a base 64 object
#We need to send a request body grant_type, that is client_credentials( clientid/clientsecret)
#The data must be encoded in application/x-www-form-urlencoded format
#and we need to send it to accounts.spotify.com/api/token
def get_token():
  #authorization string to encode in base64
  auth_string = client_id + ":" + client_secret
  auth_bytes = auth_string.encode("utf-8")
  auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
  #Return a base64 object that get converted in a string

  #URL we want to make request to 
  url = "https://accounts.spotify.com/api/token"
  #Headers associated to request to do to url 
  headers = {
      "Authorization":"Basic " + auth_base64,
      "Content-Type": "application/x-www-form-urlencoded"
  }

  data = {"grant_type": "client_credentials"}
  #URL+headers+data: we can formulate the request (post request)
  result = post(url, headers = headers, data = data)
  #We've return a json data in a field called content
  #Convert json data in python dict
  #Loads, load from string
  #Parse token, stored in a field of dict called "access_token"
  json_result = json.loads(result.content)
  token = json_result["access_token"]
  return token





#now we have access to auth token
#Once we have an authorization token, we should use it for every request we want to make to spotify web api



In [ ]:

#define authorization header for future requests
def get_auth_header(token): 
  return{"Authorization": "Bearer "+ token}



In [ ]:

#Must match the one in the application in dashboard
#Credentials to access the Spotify Music Data
#manager = SpotifyClientCredentials(client_id,client_secret)
#sp = spotipy.Spotify(client_credentials_manager=manager)

#search for an artist, get the id, get the songs of the artists




#Function to search for an artist and get its top tracks 
#The console shows what the url should be to get specific information
#
#To get an artist top track, i should know the artist id. 
#So i should first look for an artist, get its id and then use it to get the top tracks

#Search: we should construct the query specifying we're searching for an artist.
def search_for_song(token, artist_name, song_name):
  url = 'https://api.spotify.com/v1/search'
  headers = get_auth_header(token)
  full = song_name +" "+artist_name
  query = f"?q={full}&type=track&limit=1"
  #limit= 1 : gives only the first result ! 
  query_url = url + query
  result = get(query_url, headers = headers)
  json_result = json.loads(result.content)
  #Printing the artists 
  print("Artist(s):")
  for item in json_result["tracks"]["items"][0]["artists"]:
    print(item["name"])
  print("Title:")
  print(json_result["tracks"]["items"][0]["name"])
  print("Is it what you were looking for?")
  if len(json_result) == 0: 
    print("No artist with this name exists")
    return None
  return json_result["tracks"]["items"][0]["id"]


def search_for_songitems(artist_name, song_name):
  token = get_token()
  url = 'https://api.spotify.com/v1/search'
  headers = get_auth_header(token)
  full = song_name +" "+artist_name
  query = f"?q={full}&type=track&limit=1"
  query_url = url + query
  result = get(query_url, headers = headers)
  json_result = json.loads(result.content)
  if len(json_result) == 0: 
    print("No artist with this name exists")
    return None
  return json_result["tracks"]["items"]



#useless, from tutorial
def get_songs_by_artist(token, artist_id): 
  url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
  headers = get_auth_header(token)
  result = get(url, headers=headers)
  json_result = json.loads(result.content)["tracks"]
  return json_result

#taking audio features given id of song
def get_tempo_ts(artist, song): 
  
  token = get_token()
  id = search_for_song(token, artist, song)
  url = f"https://api.spotify.com/v1/audio-features/{id}"
  headers = get_auth_header(token)
  result = get(url, headers=headers)
  json_result = json.loads(result.content)
  length = json_result["duration_ms"]
  danceability = json_result["danceability"]
  acousticness = json_result["acousticness"]
  energy = json_result["energy"]
  instrumentalness = json_result["instrumentalness"]
  liveness= json_result["liveness"]
  valence = json_result["valence"]
  loudness = json_result["loudness"]
  spechiness = json_result["speechiness"]
  tempo = json_result["tempo"]
  time_signature = json_result["time_signature"]
  return tempo, time_signature
  #NUOVO!!! possibili cose da aggiornare nel mood retriaval!! 
  

#Index(['length', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       #'liveness', 'valence', 'loudness', 'speechiness', 'tempo'],



  return [length, danceability, acousticness, energy, instrumentalness, liveness, valence, loudness, spechiness, tempo]

#useless, we can get playlist with playlists id
def get_users_playlists(token, user_id): 
  url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
  headers = get_auth_header(token)
  result = get(url, headers=headers)
  json_result = json.loads(result.content)
  return json_result

def get_playlist_tracks(token, playlist_id):
  url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
  headers = get_auth_header(token)
  result = get(url, headers=headers)
  json_result = json.loads(result.content)
  return json_result



def search_for_artist(token, artist_name):
  url = "https://api.spotify.com/v1/search"
  headers = get_auth_header(token)
  #Create a query
  #artist_name, string i'm looking, but then i should specify the type
  #Get only the first artist it comes up
  query = f"?q={artist_name}&type=artist&limit=1"
  query_url =url + query
  result= get(query_url, headers=headers)
  json_result= json.loads(result.content)["artists"]["items"]
  if len(json_result)==0:
    print("No artist with this name")
    return None
  #Ritorniamo il primo elemento di items
  return json_result[0]


  

#labels: calm, energic, happy, sad
 


#da playlist : https://open.spotify.com/playlist/37i9dQZF1DWTC99MCpbjP8 
#Vengono prese le prime 100 canzoni per ora va più che bene 
#tracks_calm = get_playlist_tracks(token, "37i9dQZF1DWTC99MCpbjP8")

#tracks_energic = get_playlist_tracks(token, "37i9dQZF1DWZVAVMhIe3pV")
#result = search_for_artist(token, "ACDC")
#print(result["name"])

#artist_id = result["id"]
#songs = get_songs_by_artist(token, artist_id)
#features_calm = []
#features_energic = []
#features_happy = []
#features_sad= []

#for t in tracks_calm["items"]:

#  song_id = t["track"]["id"]
#  features_calm.append(get_audio_features_given_id(token, song_id))






Importing some cookie code from spotify 


In [ ]:
from syrics.api import Spotify
sp_dc = "AQBrzklc4elr8yh9O7KmGEsN8yLwaq1BQIN-5oqllG0m1ZZUgtGliMsRXhpHcPm4IhYTB4NL8HyZ1JWBg2HM6Zt8RDUEXzgSWytzj4jeAH9wKuxL4iiaC1K-7J_AyZZei3I-ZavpfMfhDrDY2sFapF4YOlKyjmOS"
sp = Spotify(sp_dc)

In [ ]:
#AQBrzklc4elr8yh9O7KmGEsN8yLwaq1BQIN-5oqllG0m1ZZUgtGliMsRXhpHcPm4IhYTB4NL8HyZ1JWBg2HM6Zt8RDUEXzgSWytzj4jeAH9wKuxL4iiaC1K-7J_AyZZei3I-ZavpfMfhDrDY2sFapF4YOlKyjmOS

#FINDING SP_DC: https://github.com/akashrchandran/syrics/wiki/Finding-sp_dc 
def search_on_spotify(artist, song): 
  token = get_token()
  print("id printing inside search_on_spotify function")
  id = search_for_song(token, artist, song)
  print(id)
  lrc= sp.get_lyrics(id)
  if lrc==None:
    raise Exception("The song has no lyrics or doesn't exist")
  lyrics = []

  for line in range(len(lrc["lyrics"]['lines'])): 
    time = int(lrc["lyrics"]['lines'][line]['startTimeMs'])/1000.0
    words = lrc["lyrics"]['lines'][line]['words']
    lyrics.append({'text' : words, 'start' : str(time) })
  return lyrics




